# Data collection and transmission  project
_Done by: Nasserallah EL MASSAOUDI - Rim BOUKBA
## This first notebook will be devoted to data collection


This project aims to utilize what we learned during the course _Data collection and transmission_ in order to perform sentiment analysis on data related to the BITCOIN currency that was harvested from twitter.

In [1]:
#Importing necessary libraries
import tweepy
from tweepy import  Stream
import sys
import time
import json
from datetime import datetime
import string
import pandas as pd
import utilities

# Data Collection

The data we will use for this project is streamed data from the Twitter API, corresponding to tweets related to BITCOIN. We collected tweets that had #BTC and #BITCOIN. We used the logic and the functions convert_valid() and format_filename() from the courses, as we had to reconstruct the MyListener() function to fit what we need.

In [2]:
consumer_key = ''
consumer_secret = ''
access_token = ''
access_secret = ''

"consumer_key = ''\nconsumer_secret = ''\naccess_token = ''\naccess_secret = ''"

In [3]:
auth = tweepy.OAuthHandler(consumer_key,consumer_secret,access_token,access_secret)

In [4]:
api = tweepy.API(auth)

In [5]:
query = ["#BTC","#BITCOIN"]

In [6]:
def convert_valid(one_char):
    valid_chars = "-_.%s%s" % (string.ascii_letters, string.digits)
    if one_char in valid_chars:
        return one_char
    else:
        return '_'

In [7]:
def format_filename(fname):
    return ''.join(convert_valid(one_char) for one_char in fname)

In [8]:
class MyListener(Stream):
    def __init__(self, fname,ck,cs,at,ats):
        super(MyListener,self).__init__(ck,cs,at,ats)
        safe_fname = format_filename(fname)
        self.outfile = "stream_%s.jsonl" % safe_fname

    def on_data(self, data):
        try:
            # Write code where you write your data at the end of your file
            data = data.decode("utf-8")
            with open(self.outfile,"a") as f:
                f.write(data)
                f.write("\n")

        except BaseException as e:
            sys.stderr.write("Error on_data: {}\n".format(e))
            time.sleep(5)
            return True
        
        
    def on_error(self, status):
        if status == 420:
            sys.stderr.write("Rate limit exceeded\n".format(status))
            return False
        else:
            sys.stderr.write("Error {}\n".format(status))
            return True

In [9]:
streamer = MyListener(" ".join(query),consumer_key,consumer_secret,access_token,access_secret)

In [ ]:
streamer.filter(track=query,languages=["en"])

In [11]:
streamer.disconnect()

# Data preprocessing
In the following few blocks, we will process the data that we have in order to create structure that we will further use to perform sentiment analysis on.

In [ ]:
result = []
with open("stream__BTC__BITCOIN.jsonl","r") as file:
    for tweet in file:
        result.append(json.loads(tweet))
p.pprint(result)

In [ ]:
result_clean = []
for tweet in result:
    x = {}
    x["user_followers"] = tweet["user"]["followers_count"]
    date = datetime.strptime(tweet["created_at"],'%a %b %d %X %z %Y')
    date = datetime.strftime(date,"%Y/%m/%d-%H")
    x["date"] = date
    x["text"] = tweet["text"]
    result_clean.append(x)
result_clean

In [ ]:
data = pd.DataFrame(result_clean)
data.head()

In [ ]:
data["date"] = pd.to_datetime(data["date"])
data.head()

In [ ]:
filtre_data = data[data["user_followers"]> 50000] #keeping users with a high followers count only, as they are more likely to have influence

# Remember to fix the file name

In [ ]:
filtre_data.to_csv("Finale.csv",index=False)

# Data Encryption
In this section we will encrypt our files for security measures using AES-GCM. The functions used to generate the key and encrypt the file can be found in the file utilities.py in the current directory.


In [ ]:
f_data = pd.read_csv("Finale.csv")
f_data

In [ ]:
#the utilities file contains fuction for decrypting/encrypting csv files, and generating private keys

key = utilities.Gen_Key_File()
utilities.Enc_csv("Finale.csv",key)

Please refer to the _DataProcessing.ipynb_ file